In [1]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers, ctransformers
from langchain.chains import RetrievalQA

import chainlit as cl

from langchain.text_splitter import RecursiveCharacterTextSplitter 

## check GGPU access
imnpot torch 
print(torch.cuda.__is_available__)
1+1

In [2]:
import torch
torch.cuda.is_available()

False

In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
#     llm_gpu = llm.to(device)

cpu


In [4]:
# pip install sentence_transformers

In [5]:
#### requirements.txt
# langchain==0.0.247
# chainlit==0.6.1
# pypdf==3.14.0
# faiss-cpu == 1.7.4
# transformers==4.31.0
# ctransformers #ctransformers-0.2.16-0.2.16
# sentence_transformers
# torch #'2.0.1+cpu'

# #accelerate
# #bitsandbytes

In [6]:
import torch
torch.__version__

'2.0.1+cpu'

In [7]:
import torch
torch.cuda.is_available()

False

In [8]:
# import cuda
# cuda.__version__

In [9]:
# device = torch.device"cpu'AttributeError

In [10]:
!nvidia-smi

Mon Sep  4 19:56:18 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.99                 Driver Version: 536.99       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090      WDDM  | 00000000:01:00.0  On |                  N/A |
| 33%   24C    P8              15W / 245W |  10347MiB / 24576MiB |      7%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [11]:
# pip install langchain
import langchain
langchain.__version__

'0.0.270'

In [12]:
# pip install chainlit
cl.__version__

'0.6.2'

In [13]:
# pip install sentence_transformers

In [16]:
# pip install pypdf
import pypdf
pypdf.__version__

'3.15.5'

In [22]:
# !pip install faiss-cpu
# import faiss-cpu
#faiss-cpu == 1.7.4

In [24]:
# pip install transformers

In [25]:
import transformers
transformers.__version__

'4.31.0'

In [26]:
# pip install ctransformers

In [27]:
# import ctransformers
# ctransformers.__version__

In [28]:
# !nvidia-smi

## Saving the PDF as a FAISS vector store 

In [30]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 

DATA_PATH = 'data/'
DB_FAISS_PATH = 'vectorstore/db_faiss'

# Create vector database
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#                                        model_kwargs={'device': 'cpu'})

    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)

create_vector_db()

2023-09-04 19:59:28 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
2023-09-04 19:59:28 - Use pytorch device: cpu


Batches:   0%|          | 0/224 [00:00<?, ?it/s]

2023-09-04 20:03:46 - Loading faiss with AVX2 support.
2023-09-04 20:03:46 - Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'")
2023-09-04 20:03:46 - Loading faiss.
2023-09-04 20:03:46 - Successfully loaded faiss.


## Loading the model 

In [31]:
DATA_PATH = 'data/'
DB_FAISS_PATH = 'vectorstore/db_faiss'

In [32]:


# Create vector database
def create_vector_db():
    loader = DirectoryLoader(DATA_PATH,
                             glob='*.pdf',
                             loader_cls=PyPDFLoader)

    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,
                                                   chunk_overlap=50)
    texts = text_splitter.split_documents(documents)

    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
#                                        ,model_kwargs={'device': 'cpu'})

    db = FAISS.from_documents(texts, embeddings)
    db.save_local(DB_FAISS_PATH)

# create_vector_db()

In [33]:
# embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
#                                        model_kwargs={'device': 'cpu'})

In [34]:
custom_prompt_template = """Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

def set_custom_prompt():
    """
    Prompt template for QA retrieval for each vectorstore
    """
    prompt = PromptTemplate(template=custom_prompt_template,
                            input_variables=['context', 'question'])
    return prompt

In [35]:
#Retrieval QA Chain
def retrieval_qa_chain(llm, prompt, db):
    qa_chain = RetrievalQA.from_chain_type(llm=llm,
                                       chain_type='stuff',
                                       retriever=db.as_retriever(search_kwargs={'k': 4}),
                                       return_source_documents=True,
                                       chain_type_kwargs={'prompt': prompt}
                                       )
    return qa_chain

In [37]:
#Loading the model
def load_llm():
    # Load the locally downloaded model here
#     llm = CTransformers(
# #         model = "C:/Users/vibud/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
# #         model = "http://127.0.0.1:8888/tree/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
#         model = "/mnt/c/Users/vibud/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
#         model_type="llama",
#         max_new_tokens = 512,
#         temperature = 0.5
#     )
    llm = complex_model_load("TheBloke_Llama-2-13B-chat-GPTQ")
    return llm

In [38]:
#QA Model Function
def qa_bot():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
    db = FAISS.load_local(DB_FAISS_PATH, embeddings)
    llm = load_llm()
    qa_prompt = set_custom_prompt()
    qa = retrieval_qa_chain(llm, qa_prompt, db)

    return qa

In [39]:
#output function
def final_result(query):
    qa_result = qa_bot()
    response = qa_result({'query': query})
    return response

In [41]:
query = "My stomach has been feeling uncomfortable for many days. I have to go poop multiple times a day and I am feel slight stomach ache. What could be the possible cause of my problem?"

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
db = FAISS.load_local(DB_FAISS_PATH, embeddings)
llm = load_llm()
qa_prompt = set_custom_prompt()
qa_prompt

2023-09-04 20:08:53 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


PromptTemplate(input_variables=['context', 'question'], output_parser=None, partial_variables={}, template="Use the following pieces of information to answer the user's question.\nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext: {context}\nQuestion: {question}\n\nOnly return the helpful answer below and nothing else.\nHelpful answer:\n", template_format='f-string', validate_template=True)

In [ ]:
qa_result = retrieval_qa_chain(llm, qa_prompt, db)
response = qa_result({'query': query})
response

In [ ]:
response['result']

In [ ]:
!nvidia-smi

## Try to load the model to the GPU

In [ ]:
# #Loading the model
# def load_llm():
#     # Load the locally downloaded model here
#     llm = CTransformers(
#         model = "C:\\Users\\vibud\\Desktop\\Github\\llama-2-experiments\\Models\\llama-2-7b-chat.ggmlv3.q8_0.bin",
#         model_type="llama",
#         max_new_tokens = 512,
#         temperature = 0.5
#     )
#     llm_gpu = llm.to('cuda')
#     return llm


#Loading the model
def load_llm():
    # Load the locally downloaded model here
    llm = CTransformers(
#         model = "C:/Users/vibud/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
#         model = "http://127.0.0.1:8888/tree/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
        model = "/mnt/c/Users/vibud/Desktop/Github/llama-2-experiments/Models/llama-2-7b-chat.ggmlv3.q8_0.bin",
        model_type="llama",
        max_new_tokens = 512,
        temperature = 0.5,
        gpu_layers = 50
    )
#     device = "cuda:0" if torch.cuda.is_available() else "cpu"
#     print(device)
#     llm_gpu = llm.to(device)
    return llm

In [ ]:
query = "My stomach has been feeling uncomfortable for many days. I have to go poop multiple times a day and I am feel slight stomach ache. What could be the possible cause of my problem?"

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                       model_kwargs={'device': 'cpu'})
db = FAISS.load_local(DB_FAISS_PATH, embeddings)
llm = load_llm()
qa_prompt = set_custom_prompt()
qa_result = retrieval_qa_chain(llm, qa_prompt, db)
response = qa_result({'query': query})
response

2023-08-01 23:12:36 - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

{'query': 'My stomach has been feeling uncomfortable for many days. I have to go poop multiple times a day and I am feel slight stomach ache. What could be the possible cause of my problem?',
 'result': 'Antidiarrestabdomestic ulcervom/p style="Antidiarrhile stom your bowelcome on the symptoms are you don\'\nDiarrested, or get medical attention should go to keep reading and bloating a  I can help- Antidiarrhese drugs\nI cannot provide information provided below\nAntidiarrestabdomesticated with antidiarrhileusually moves into the symptoms: If diarrhe problem that lastsometimes,\nDiarrhe diarrhe diarrhe diarrestabdomestic ulceremesis that may have you should consult a)\nI don\'\nIf your doctor or if you don\'The information provided belowHelpful answer_xCertainly controlled tripped after discussions  Hello there are you don\'mildi can vary depending on occasion when someone in addition of a diarrested as soon as soon as soon as soon as soon as soon as soon as soon,\nAntidiarrestabdomesti

## Result

In [43]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)
#     llm_gpu = llm.to(device)

cuda:0


- Cuda is available 
- But llm.to(device) does not work 
- llm is a CTransformer object not a transformer object 
- [GPU support link](https://github.com/marella/ctransformers#gpu)
- [Colab support link](https://colab.research.google.com/drive/1Ihn7iPCYiqlTotpkqa1tOhUIpJBrJ1Tp#scrollTo=8Hu74s0OdmQ3)